<a href="https://colab.research.google.com/github/klutzydrummer/Python_Projects/blob/main/preprocess_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Original article code was ported from](https://towardsdatascience.com/stock-predictions-with-state-of-the-art-transformer-and-time-embeddings-3a4485237de6)

In [ ]:
from pathlib import Path as path
req_flag = path('/content/req_flag')
import os, sys
if req_flag.exists() is not True:
    from google.colab import drive
    drive.mount('/content/drive')
    nb_path = '/content/drive/My Drive/Machine_Learning_Digestor/.cache'
    sys.path.append('/content/drive/My Drive/Machine_Learning_Digestor/.cache')
    !pip install pymysql tqdm cache-magic yfinance-cache seaborn ncps backtrader icecream
    !pip3 install -U ncps pytorch-lightning
    # import cache_magic
    try:
        os.symlink('/content/.cache', nb_path)
    except:
        pass
    sys.path.insert(0, nb_path)  # or append(nb_path)
    %cd '/content/drive/My Drive/Machine_Learning_Digestor/'
    req_flag.touch()

In [ ]:
if 'yf' not in globals():
    import yfinance_cache as yf

In [ ]:
from pathlib import Path
dataset_root = Path('/content/drive/MyDrive/Machine_Learning_Digestor/datasets')
train_dataset = dataset_root / 'train'
val_dataset = dataset_root / 'val'
test_dataset = dataset_root / 'test'
train_dataset.mkdir(exist_ok=True, parents=True)
val_dataset.mkdir(exist_ok=True, parents=True)
test_dataset.mkdir(exist_ok=True, parents=True)

In [ ]:
import numpy as np
import pandas as pd
import os, datetime

import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8')

import warnings
warnings.filterwarnings('ignore')


<ipython-input-4-32688035f85b>:10: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [ ]:
from pathlib import Path as path

target_stocks = ["MMM","AOS","ABT","ABBV","ACN","ATVI","ADM","ADBE","ADP","AAP","AES","AFL","A","APD","AKAM","ALK","ALB","ARE","ALGN","ALLE","LNT","ALL","GOOGL","GOOG","MO","AMZN","AMCR","AMD","AEE","AAL","AEP","AXP","AIG","AMT","AWK","AMP","ABC","AME","AMGN","APH","ADI","ANSS","AON","APA","AAPL","AMAT","APTV","ACGL","ANET","AJG","AIZ","T","ATO","ADSK","AZO","AVB","AVY","AXON","BKR","BALL","BAC","BBWI","BAX","BDX","WRB","BRK.B","BBY","BIO","TECH","BIIB","BLK","BK","BA","BKNG","BWA","BXP","BSX","BMY","AVGO","BR","BRO","BF.B","BG","CHRW","CDNS","CZR","CPT","CPB","COF","CAH","KMX","CCL","CARR","CTLT","CAT","CBOE","CBRE","CDW","CE","CNC","CNP","CDAY","CF","CRL","SCHW","CHTR","CVX","CMG","CB","CHD","CI","CINF","CTAS","CSCO","C","CFG","CLX","CME","CMS","KO","CTSH","CL","CMCSA","CMA","CAG","COP","ED","STZ","CEG","COO","CPRT","GLW","CTVA","CSGP","COST","CTRA","CCI","CSX","CMI","CVS","DHI","DHR","DRI","DVA","DE","DAL","XRAY","DVN","DXCM","FANG","DLR","DFS","DIS","DG","DLTR","D","DPZ","DOV","DOW","DTE","DUK","DD","DXC","EMN","ETN","EBAY","ECL","EIX","EW","EA","ELV","LLY","EMR","ENPH","ETR","EOG","EPAM","EQT","EFX","EQIX","EQR","ESS","EL","ETSY","EG","EVRG","ES","EXC","EXPE","EXPD","EXR","XOM","FFIV","FDS","FICO","FAST","FRT","FDX","FITB","FSLR","FE","FIS","FI","FLT","FMC","F","FTNT","FTV","FOXA","FOX","BEN","FCX","GRMN","IT","GEHC","GEN","GNRC","GD","GE","GIS","GM","GPC","GILD","GL","GPN","GS","HAL","HIG","HAS","HCA","PEAK","HSIC","HSY","HES","HPE","HLT","HOLX","HD","HON","HRL","HST","HWM","HPQ","HUM","HBAN","HII","IBM","IEX","IDXX","ITW","ILMN","INCY","IR","PODD","INTC","ICE","IFF","IP","IPG","INTU","ISRG","IVZ","INVH","IQV","IRM","JBHT","JKHY","J","JNJ","JCI","JPM","JNPR","K","KDP","KEY","KEYS","KMB","KIM","KMI","KLAC","KHC","KR","LHX","LH","LRCX","LW","LVS","LDOS","LEN","LNC","LIN","LYV","LKQ","LMT","L","LOW","LYB","MTB","MRO","MPC","MKTX","MAR","MMC","MLM","MAS","MA","MTCH","MKC","MCD","MCK","MDT","MRK","META","MET","MTD","MGM","MCHP","MU","MSFT","MAA","MRNA","MHK","MOH","TAP","MDLZ","MPWR","MNST","MCO","MS","MOS","MSI","MSCI","NDAQ","NTAP","NFLX","NWL","NEM","NWSA","NWS","NEE","NKE","NI","NDSN","NSC","NTRS","NOC","NCLH","NRG","NUE","NVDA","NVR","NXPI","ORLY","OXY","ODFL","OMC","ON","OKE","ORCL","OGN","OTIS","PCAR","PKG","PANW","PARA","PH","PAYX","PAYC","PYPL","PNR","PEP","PFE","PCG","PM","PSX","PNW","PXD","PNC","POOL","PPG","PPL","PFG","PG","PGR","PLD","PRU","PEG","PTC","PSA","PHM","QRVO","PWR","QCOM","DGX","RL","RJF","RTX","O","REG","REGN","RF","RSG","RMD","RVTY","RHI","ROK","ROL","ROP","ROST","RCL","SPGI","CRM","SBAC","SLB","STX","SEE","SRE","NOW","SHW","SPG","SWKS","SJM","SNA","SEDG","SO","LUV","SWK","SBUX","STT","STLD","STE","SYK","SYF","SNPS","SYY","TMUS","TROW","TTWO","TPR","TRGP","TGT","TEL","TDY","TFX","TER","TSLA","TXN","TXT","TMO","TJX","TSCO","TT","TDG","TRV","TRMB","TFC","TYL","TSN","USB","UDR","ULTA","UNP","UAL","UPS","URI","UNH","UHS","VLO","VTR","VRSN","VRSK","VZ","VRTX","VFC","VTRS","VICI","V","VMC","WAB","WBA","WMT","WBD","WM","WAT","WEC","WFC","WELL","WST","WDC","WRK","WY","WHR","WMB","WTW","GWW","WYNN","XEL","XYL","YUM","ZBRA","ZBH","ZION","ZTS"]

stock_datasets = {stock : {'period': 'max'} for stock in target_stocks}

In [ ]:
import json
with open("/content/drive/MyDrive/Machine_Learning_Digestor/config/ps_conn_details_preprocesser.json", "r") as f:
    ps_conn_details = json.load(f)

import pandas as pd
import pymysql
from datetime import datetime

def join_sentiment_scores(cursor, stock_symbol, yfinance_df):
    try:
        query = f"""
            SELECT publish_date, CAST(AVG(sentiment_score) AS FLOAT) as avg_sentiment_score
            FROM sentiment_scores
            WHERE stock_symbol = %s
            GROUP BY publish_date
        """
        cursor.execute(query, (stock_symbol,))
        sentiment_data = cursor.fetchall()

        sentiment_df = pd.DataFrame(sentiment_data, columns=['publish_date', 'avg_sentiment_score'])
        sentiment_df['publish_date'] = sentiment_df['publish_date'].apply(lambda x: x.date())  # Extract date from datetime

        yfinance_df['date'] = yfinance_df['Date'].apply(lambda x: x.date())  # Extract date from datetime
        merged_df = pd.merge(yfinance_df, sentiment_df, how='left', left_on='date', right_on='publish_date')
        merged_df.drop(columns=['publish_date'], inplace=True)

        # Fill NaN values in avg_sentiment_score with 0
        merged_df['avg_sentiment_score'].fillna(0, inplace=True)

        return merged_df

    except Exception as e:
        print(f"An error occurred: {e}")
        yfinance_df['avg_sentiment_score'] = 0
        return yfinance_df

In [ ]:
from pathlib import Path as path

connection = pymysql.connect(**ps_conn_details)
cursor = connection.cursor()

## Calculate normalized percentage change of all columns add helper columns

In [ ]:
from hashlib import sha256
import numpy as np
import pandas as pd
import pymysql

class StockDBManager:
    def __init__(self, conn_details):
        self.conn_details = conn_details
        self.conn = None

    def connect(self):
        self.conn = pymysql.connect(**self.conn_details)

    def close(self):
        if self.conn:
            self.conn.close()

    def upload_dataframe(self, df, table_name='StockData', overwrite=False):
        if self.conn is None:
            print("Not connected to database.")
            return

        cursor = self.conn.cursor()

        for index, row in df.iterrows():
            columns = ', '.join(row.index)
            values = ', '.join([f"'{str(value)}'" for value in row])

            sql_insert_query = f"INSERT IGNORE INTO {table_name} ({columns}) VALUES ({values})"

            if overwrite:
                sql_insert_query = f"REPLACE INTO {table_name} ({columns}) VALUES ({values})"

            try:
                cursor.execute(sql_insert_query)
            except Exception as e:
                print(f"Failed to insert row {index}: {e}")

        self.conn.commit()
        print(f"Uploaded DataFrame to table {table_name}")

    def check_stock_symbol_exists(self, stock_symbol, table_name='StockData'):
            if self.conn is None:
                print("Not connected to database.")
                return False

            try:
                with self.conn.cursor() as cursor:
                    # Define the SQL query to check if the stock_symbol exists
                    sql = f"SELECT COUNT(*) FROM {table_name} WHERE stock_symbol = %s"

                    # Execute the query with the stock_symbol as a parameter
                    cursor.execute(sql, (stock_symbol,))

                    # Fetch the result (count of matching rows)
                    result = cursor.fetchone()

                    # Check if there are any matching rows
                    if result[0] > 0:
                        return True  # Stock symbol exists in the table
                    else:
                        return False  # Stock symbol does not exist in the table

            except pymysql.Error as e:
                print(f"Error: {e}")
                return False  # An error occurred, return False

    def worker(self, stock_symbols_chunk, split_ratios=(0.7, 0.2, 0.1)):
        conn = self.connect()
        cursor = conn.cursor()

        try:
            for symbol in stock_symbols_chunk:
                cursor.execute(f"SELECT unique_id FROM StockData WHERE stock_symbol = %s ORDER BY Date", (symbol,))
                ids = [item[0] for item in cursor.fetchall()]

                total_count = len(ids)
                train_count = math.floor(total_count * split_ratios[0])
                val_count = math.floor(total_count * split_ratios[1])

                for idx, unique_id in enumerate(ids):
                    if idx < train_count:
                        label = 'train'
                    elif idx < train_count + val_count:
                        label = 'val'
                    else:
                        label = 'test'

                    update_query = f"UPDATE StockData SET dataset_label = %s WHERE unique_id = %s"
                    cursor.execute(update_query, (label, unique_id))

            conn.commit()

        except pymysql.Error as e:
            print(f"Error: {e}")
        finally:
            conn.close()

    def update_dataset_labels_threaded(self, n_workers=3, split_ratios=(0.7, 0.2, 0.1)):
        if sum(split_ratios) != 1:
            raise ValueError('split ratio must sum to 1')
        conn = self.connect()
        cursor = conn.cursor()
        cursor.execute(f"SELECT DISTINCT stock_symbol FROM StockData")
        stock_symbols = [item[0] for item in cursor.fetchall()]
        conn.close()

        # Split the list of stock_symbols among the workers
        chunk_size = len(stock_symbols) // n_workers
        stock_symbols_chunks = [stock_symbols[i:i + chunk_size] for i in range(0, len(stock_symbols), chunk_size)]

        with ThreadPoolExecutor(max_workers=n_workers) as executor:
            executor.map(self.worker, stock_symbols_chunks, split_ratios)


def preprocess_dataframe(df, current_stock, split_ratios=(0.7, 0.2, 0.1)):
    if 'Date' not in df.columns:
        raise ValueError("No 'Date' column found.")
    # Calculate split indices
    num_rows = len(df)
    train_end = int(split_ratios[0] * num_rows)
    val_end = train_end + int(split_ratios[1] * num_rows)

    # Assign dataset labels
    df['dataset_label'] = ''
    df.loc[:train_end, 'dataset_label'] = 'train'
    df.loc[train_end:val_end, 'dataset_label'] = 'val'
    df.loc[val_end:, 'dataset_label'] = 'test'

    # Check for unlabeled data
    if df['dataset_label'].eq('').any():
        raise ValueError("Found unlabeled data rows. Every row should have a dataset label.")

    date_col = pd.to_datetime(df['Date'])
    df['year'] = date_col.dt.year
    df['month'] = date_col.dt.month
    df['day'] = date_col.dt.day
    df['weekday'] = date_col.dt.weekday
    df['hour'] = date_col.dt.hour
    df['minute'] = date_col.dt.minute
    df['second'] = date_col.dt.second

    unique_id = int(sha256(current_stock.encode()).hexdigest(), 16) % (10 ** 8)
    df['unique_id'] = unique_id / 10 ** 8
    df['stock_symbol'] = current_stock

    df['target'] = df['Close'].shift(1)
    df['SMA'] = df['Close'].rolling(window=200).mean()
    exp12 = df['Close'].ewm(span=12, adjust=False).mean()
    exp26 = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = exp12 - exp26
    df.dropna(inplace=True)

    df['Log_Returns_Close'] = np.log(df['Close'] / df['Close'].shift(1)).replace([np.inf, -np.inf], 0)
    df.dropna(inplace=True)

    df.replace([np.inf, -np.inf], 0, inplace=True)
    df.dropna(how='any', axis=0, inplace=True)

    for col in ['Open', 'High', 'Low', 'Close', 'Volume', 'target', 'SMA', 'MACD', 'Log_Returns_Close']:
        df[col] = df[col].pct_change()

    df.dropna(inplace=True)
    df.drop(columns=['date'], inplace=True, errors='ignore')

    int_cols = ['year', 'month', 'day', 'weekday', 'hour', 'minute', 'second']
    float_cols = [col for col in df.columns if col not in int_cols + ['Date', 'stock_symbol', 'dataset_label']]

    df[int_cols] = df[int_cols].astype('int64')
    df[float_cols] = df[float_cols].astype('float32')

    df.replace([np.inf, -np.inf], 0, inplace=True)

    return df

def split_dataset(df):
    df_train = df.loc[df['dataset_label'] == 'train']
    df_val = df.loc[df['dataset_label'] == 'val']
    df_test = df.loc[df['dataset_label'] == 'test']
    return df_train, df_val, df_test

# Process data
window_size = 365
overwrite = False
update = True
for current_stock, stock_metadata in stock_datasets.items():
    save_path_train_dataset = train_dataset / f'{current_stock}.parquet'
    save_path_val_dataset = val_dataset / f'{current_stock}.parquet'
    save_path_test_dataset = test_dataset / f'{current_stock}.parquet'
    if save_path_test_dataset.exists():
        pass
    period = stock_metadata['period']
    stock_ticker = yf.Ticker(current_stock)
    try:
        stock_ticker.verify_cached_prices(correct=True)
    except:
        pass
    try:
        df = stock_ticker.history(interval='1d', period=period)# , start=start_date, end=end_date)
    except:
        continue
    selected_columns = ["Close", "High", "Low", "Open", "Volume"]
    df = df[selected_columns]

    # Filter out dates before 1900
    df = df[df.index.year > 1900]

    # Reset the datetime index
    df = df.reset_index()

    # Rename the index to 'Date'
    df.rename(columns={'index': 'Date'}, inplace=True)

    connection = pymysql.connect(**ps_conn_details)
    cursor = connection.cursor()
    df = join_sentiment_scores(cursor, current_stock, df)

    # Replace 0 to avoid dividing by 0 later on
    df['Volume'].replace(to_replace=0, method='ffill', inplace=True)

    df = preprocess_dataframe(df, current_stock)

    df_train, df_val, df_test = split_dataset(df)
    df_train.to_parquet(str(save_path_train_dataset))
    df_val.to_parquet(str(save_path_val_dataset))
    df_test.to_parquet(str(save_path_test_dataset))

Streaming output truncated to the last 5000 lines.
1980-03-20 00:00:00-05:00    True 2023-09-17 21:16:27.088774+00:00      True   
1980-03-21 00:00:00-05:00    True 2023-09-17 21:16:27.088774+00:00      True   
...                           ...                              ...       ...   
2023-09-11 00:00:00-04:00    True 2023-09-17 21:16:27.088774+00:00      True   
2023-09-12 00:00:00-04:00    True 2023-09-17 21:16:27.088774+00:00      True   
2023-09-13 00:00:00-04:00    True 2023-09-17 21:16:27.088774+00:00      True   
2023-09-14 00:00:00-04:00    True 2023-09-17 21:16:27.088774+00:00      True   
2023-09-15 00:00:00-04:00   False 2023-09-17 21:16:27.088774+00:00      True   

                                           LastDivAdjustDt  \
Date                                                         
1980-03-17 00:00:00-05:00 2023-09-17 21:16:27.626164+00:00   
1980-03-18 00:00:00-05:00 2023-09-17 21:16:27.626164+00:00   
1980-03-19 00:00:00-05:00 2023-09-17 21:16:27.626164+00:00  

ERROR:yfinance:BRK.B: No data found, symbol may be delisted


Streaming output truncated to the last 5000 lines.
1980-03-19 00:00:00-05:00  2271600        0.0           0.0       True   
1980-03-20 00:00:00-05:00   805042        0.0           0.0       True   
1980-03-21 00:00:00-05:00   814410        0.0           0.0       True   
...                            ...        ...           ...        ...   
2023-09-11 00:00:00-04:00  1556300        0.0           0.0      False   
2023-09-12 00:00:00-04:00  1270300        0.0           0.0      False   
2023-09-13 00:00:00-04:00  1248300        0.0           0.0      False   
2023-09-14 00:00:00-04:00  1837800        0.0           0.0      False   
2023-09-15 00:00:00-04:00  1611400        0.0           0.0      False   

                           Final?                        FetchDate  C-Check?  \
Date                                                                           
1980-03-17 00:00:00-05:00    True 2023-09-17 21:37:24.433495+00:00      True   
1980-03-18 00:00:00-05:00    True 2023-09-

In [ ]:
import os
import glob
from pathlib import Path

dataset_root = Path('/content/drive/MyDrive/Machine_Learning_Digestor/datasets')
train_dataset = dataset_root / 'train'
val_dataset = dataset_root / 'val'
test_dataset = dataset_root / 'test'
train_dataset.mkdir(exist_ok=True, parents=True)
val_dataset.mkdir(exist_ok=True, parents=True)
test_dataset.mkdir(exist_ok=True, parents=True)

def find_parquet_files(directory_path):
    return glob.glob(os.path.join(directory_path, '*.parquet'))


train_dataset_parquets = find_parquet_files(train_dataset)
val_dataset_parquets = find_parquet_files(val_dataset)
test_dataset_parquets = find_parquet_files(test_dataset)